In [11]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")
response = llm.invoke("What is llm")
print(response)


LLM stands for Large Language Model, which refers to a type of artificial intelligence (AI) model that has been trained on a massive amount of text data. These models are designed to understand and generate human-like language, with the goal of improving natural language processing (NLP) capabilities.

Large Language Models typically consist of millions or even billions of neural network parameters, which allow them to learn complex patterns in language and generate coherent, context-aware text. They are trained on vast amounts of text data, such as books, articles, and websites, and can be used for a wide range of applications, including:

1. **Language translation**: LLMs can translate text from one language to another with high accuracy.
2. **Text summarization**: LLMs can summarize long pieces of text into concise, meaningful summaries.
3. **Question answering**: LLMs can answer questions based on the information they've been trained on.
4. **Content generation**: LLMs can generate

In [12]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyPDFDirectoryLoader


# loader = DirectoryLoader('train_files', glob="**/*.csv", loader_cls=CSVLoader)
# loader = CSVLoader(file_path="train_files/tweets_sentTransformer_2.csv")
# docs = loader.load()
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
docs

[Document(page_content='Research Journal of Chemical Sciences______________ _______________________________Vol. 1 (2) May (2011 ) \n                                    Res.J.Chem.Sci  \n \n135 \n Review Paper \nTargeted drug delivery system: A Review \nGupta Manish and Sharma Vimukta \nBM College of Pharmaceutical Education and Research , Indore, INDIA \n \nAvailable online at: www.isca.in  \n(Received 17 th   March 2011, revised 04 th  May 2011, accepted 05 th  May 2011) \n \nAbstract \nTargeted drug delivery is a method of delivering me dication to a patient in a manner that increases th e \nconcentration of the medication in some parts of th e body relative to others. Targeted drug delivery \nseeks to concentrate the medication in the tissues of interest while reducing the relative concentrati on of \nthe medication in the remaining tissues. This impro ves efficacy of the while reducing side effects. It  is \nvery difficult for a drug molecule to reach its des tination in the comple

In [13]:
print(docs[1])

page_content='Research Journal of Chemical Sciences______________ _______________________________Vol. 1 (2) May (2011 ) \n                                    Res.J.Chem.Sci  \n \n136 \n strain, impurities), the presence of an interface \nbetween different semiconductor materials (e.g. in \ncore-shell nanocrystal systems), the presence of th e \nsemiconductor surface (e.g. semiconductor \nnanocrystal), or a combination of these. Quantum \ndots are particularly significant for optical \napplications due to their theoretically high quantu m \nyield. The ability to tune the size of quantum dots  is \nadvantageous for many applications and it is one of  \nthe most promising candidates for use in solid-stat e \nquantum computation and diagnosis , drug delivery, \nTissue engineering, catalysis, filtration and also \ntextiles technologies. \n \n                         \n \n \n \nTransdermal Approach: Transdermal drug delivery \nsystem is topically administered medicaments in the  \nform of pa

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model='llama3')

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pickle

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

with open("vector", "wb") as f:
    pickle.dump(vector, f)

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
retriever = vector.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
document_chain = create_history_aware_retriever(llm, retriever, prompt)

In [17]:
from langchain.chains import create_retrieval_chain
import pickle

with open("vector", "rb") as f:
    vector = pickle.load(f)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": """" Explain the target drug delivery and why it generated a great intrest?"""})
print(response['answer'])

# LangSmith offers several features that can help with testing:...

[Document(page_content='Research Journal of Chemical Sciences______________ _______________________________Vol. 1 (2) May (2011 ) \n                                    Res.J.Chem.Sci  \n \n138 \n  \n-1822 (2004)  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n References \n1. Chien Y.W., Novel drug delivery systems, Drugs \nand the Pharmaceutical Sciences,   50 ,     New York, \n797, 992 (2008)   \n \n2. Allen T. M. and Cullis P. R., Drug Delivery \nSystems, Entering the Mainstream Science, 1818 \n \n \n3. Nacht S.  and Kantz M,  A.,  Novel  Topical  \nProgrammable Delivery System, Topical Drug \nDelivery Systems, 299-325 (1992)  \n \n4. Won R., Method for delivering an active \ningredient by controlled time release utilizing a \nnovel delivery vehicle, 825 (1987)', metadata={'source': 'pdfs/drug.pdf', 'page': 3}), Document(page_content='wit h the vast database of different studies, the \nscience of site specific or targeted deliver